# Detailed comparison between deep and non-deep learning

## By Brecht De Boeck - on behalf of Group 10

The difference between deep learning and non-deep learning is very simple: Any neural network with at least three layers is considered deep learning.

For this part of our assignment, I'll be creating two separate image classification models, one deep learning and one non-deep, and comparing the two based on both the speed at which they make a prediction, and their overall performance.

### Creating the non-deep learning models

In [123]:
import numpy as np
import pandas as pd
import cv2 as cv
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [124]:
#Read the basic data from the labeled dataset
data = pd.read_csv("data/swarmIoT_full_labeled_dataset.csv")
images = data["image"].str.split("-", n = 1, expand = True)
data["image"] = images[1]

Neural networks are the most obvious choice when it comes to image classification, but these generally fall under deep learning. We could try using a neural network without any hidden layers, where input and output are directly connected.

However, we have seen a lot of non-deep learning models throughout this course. Sure, none of them were explicitly used for image classification, but that doesn't mean we can't try using one of them for it. But then the question remains: which algorithm do we choose?

Our options:
- kNN: Nope. Not using this one. Not only is it way too slow, but it's pretty much undeployable, seeing as there isn't a real model that's created.
- Support Vector Machines: While an interesting choice, these kinds of models don't do very well in the way of speed and scaleability. Especially in the case of that first one, we don't want to give our neural network an unfair advantage.
- Decision Trees: Seems to be our best fit. Though it cannot actually see the images, but only their raw values, it is capable of making decisions based on the given input, similar to how a deep learning neural network would do it.
- Logistic Regression: As bizarre as it might seem, we can also use Logistic Regression if we reshape the dimensions of our training and test set to consist of at most two dimensions.

For this task, I'll be making both a Decision Tree and a Logistic Regression model. The Decision Tree (at least from my perspective) seems to lie the closest to emulating a neural network, while for Logistic Regression, I'm simply curious to find out what results the model might give us.

And finally, as discussed, I'll be making a neural network without any hidden layers. All of these models will be compared to our deep learning models made in the next chapter of this notebook.

#### Extracting features from the images

The problem with decision trees is that, of course, we can't just feed raw images to our model. The decision tree expects features with differing values, which it can then use to make splits to gain as much information gain as possible. The same goes for our Logistic Regression model: we can't just expect to be able to throw in raw image data and have the model make accurate predictions. Our first order of business is thus to create a new dataset, that combines our classifiers and their images with features we extract from the images.

In [125]:
from sklearn.cluster import MiniBatchKMeans
import time
import os

In [146]:
# The following function will be used to process every image within our dataset
def extract_features_from_image(camera, partition, img):
    st = time.time()
    path = "/".join(['data','images',camera,partition,img])
    partition_path = "/".join(['data','images',camera,partition])
    image = plt.imread(path)
    
    dicto = []
    
    if image is not None:
        #Extract features using SIFT
        sift = cv.SIFT_create()
        kp, des = sift.detectAndCompute(image,None)
        for d in des:
            dicto.append(d)
        
        k = 30
        batch_size = np.size(os.listdir(partition_path)) * 3
        kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(dico)
        
        histo = np.zeros(k)
        nkp = np.size(kp)
        for d in des:
            idx = kmeans.predict([d.astype(np.double)])
            histo[idx] += 1/nkp
    return histo

Using the above method should work when working with machine learning models, but we cannot use it on all of our images. That would simply be a fool's errand, clustering the keypoints of more than 6000 images. It's a good thing our images are split amongst partitions.

I'll be using the images from Camera 3, specifically the first partition, which our group labeled ourselves and only contains 856 images. I'll be taking the first 50 images from this partition and splitting it into training and test datasets. That's all the abuse I'm willing to inflict on my poor computer, even if the resulting sample won't give us nearly as much information as when working on the full dataset or even just a simple partition.

**For the Record**: Yes, I did try getting the features of an entire partition. That attempt resulted in my computer crashing. Again. I work with what I have in this assignment, so please just bare with me.

In [148]:
sample = os.listdir("data/images/3 De nayer (Loop piste)/partition 1")
sample = data.loc[data["image"].isin(sample)].head(50)

#extract_features_from_image("3 De nayer (Loop piste)", "partition 1",sample["image"].head(1).iloc[0])
sample_feats = [extract_features_from_image("3 De nayer (Loop piste)", "partition 1", img) for img in sample["image"]]

C:\Users\Brech\anaconda3\envs\chidi\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 3072 or by setting the environment variable OMP_NUM_THREADS=9
  warnings.warn(


Init 1/3 with method k-means++
Inertia for init 1/3: 291826708.0
Init 2/3 with method k-means++
Inertia for init 2/3: 290078818.0
Init 3/3 with method k-means++
Inertia for init 3/3: 286772473.0
Minibatch step 1/100: mean batch inertia: 137716.42173288655
Minibatch step 2/100: mean batch inertia: 91830.39866856401, ewa inertia: 91830.39866856401
Minibatch step 3/100: mean batch inertia: 89250.01682988797, ewa inertia: 89250.01682988797
Minibatch step 4/100: mean batch inertia: 89442.3004474216, ewa inertia: 89442.3004474216
Minibatch step 5/100: mean batch inertia: 89046.21283550088, ewa inertia: 89046.21283550088
Minibatch step 6/100: mean batch inertia: 89473.04884279845, ewa inertia: 89473.04884279845
Minibatch step 7/100: mean batch inertia: 88998.75502006288, ewa inertia: 88998.75502006288
Minibatch step 8/100: mean batch inertia: 87933.95818179616, ewa inertia: 87933.95818179616
Minibatch step 9/100: mean batch inertia: 88829.28346020142, ewa inertia: 88829.28346020142
Minibatch 

C:\Users\Brech\anaconda3\envs\chidi\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 3072 or by setting the environment variable OMP_NUM_THREADS=9
  warnings.warn(


Init 1/3 with method k-means++
Inertia for init 1/3: 289068101.0
Init 2/3 with method k-means++
Inertia for init 2/3: 286070393.0
Init 3/3 with method k-means++
Inertia for init 3/3: 288896724.0
Minibatch step 1/100: mean batch inertia: 137437.56199138344
Minibatch step 2/100: mean batch inertia: 93037.85649165805, ewa inertia: 93037.85649165805
Minibatch step 3/100: mean batch inertia: 90028.59801615696, ewa inertia: 90028.59801615696
Minibatch step 4/100: mean batch inertia: 89278.1878997089, ewa inertia: 89278.1878997089
Minibatch step 5/100: mean batch inertia: 89844.75701503576, ewa inertia: 89844.75701503576
Minibatch step 6/100: mean batch inertia: 89119.92515245198, ewa inertia: 89119.92515245198
Minibatch step 7/100: mean batch inertia: 89154.16215770101, ewa inertia: 89154.16215770101
Minibatch step 8/100: mean batch inertia: 89421.00648220864, ewa inertia: 89421.00648220864
Minibatch step 9/100: mean batch inertia: 88220.81565188276, ewa inertia: 88220.81565188276
Minibatch 

C:\Users\Brech\anaconda3\envs\chidi\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 3072 or by setting the environment variable OMP_NUM_THREADS=9
  warnings.warn(


...

Init 1/3 with method k-means++
Inertia for init 1/3: 284515313.0
Init 2/3 with method k-means++
Inertia for init 2/3: 288693375.0
Init 3/3 with method k-means++
Inertia for init 3/3: 284312382.0
Minibatch step 1/100: mean batch inertia: 138951.20679751076
Minibatch step 2/100: mean batch inertia: 91914.11042847371, ewa inertia: 91914.11042847371
Minibatch step 3/100: mean batch inertia: 89782.06529601687, ewa inertia: 89782.06529601687
Minibatch step 4/100: mean batch inertia: 89454.8778983082, ewa inertia: 89454.8778983082
Minibatch step 5/100: mean batch inertia: 89227.42873290119, ewa inertia: 89227.42873290119
Minibatch step 6/100: mean batch inertia: 90076.80098852377, ewa inertia: 90076.80098852377
Minibatch step 7/100: mean batch inertia: 88821.30681513858, ewa inertia: 88821.30681513858
Minibatch step 8/100: mean batch inertia: 88367.68750590215, ewa inertia: 88367.68750590215
Minibatch step 9/100: mean batch inertia: 88516.90955420045, ewa inertia: 88516.90955420045
Minibatch 

C:\Users\Brech\anaconda3\envs\chidi\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 3072 or by setting the environment variable OMP_NUM_THREADS=9
  warnings.warn(


Init 1/3 with method k-means++
Inertia for init 1/3: 290026913.0
Init 2/3 with method k-means++
Inertia for init 2/3: 289403253.0
Init 3/3 with method k-means++
Inertia for init 3/3: 288397767.0
Minibatch step 1/100: mean batch inertia: 139567.68884633796
Minibatch step 2/100: mean batch inertia: 91458.42338827484, ewa inertia: 91458.42338827484
Minibatch step 3/100: mean batch inertia: 89745.25204021663, ewa inertia: 89745.25204021663
Minibatch step 4/100: mean batch inertia: 89513.92028581498, ewa inertia: 89513.92028581498
Minibatch step 5/100: mean batch inertia: 89371.16473354516, ewa inertia: 89371.16473354516
Minibatch step 6/100: mean batch inertia: 89655.70240116859, ewa inertia: 89655.70240116859
Minibatch step 7/100: mean batch inertia: 88287.18299807311, ewa inertia: 88287.18299807311
Minibatch step 8/100: mean batch inertia: 88421.81604494035, ewa inertia: 88421.81604494035
Minibatch step 9/100: mean batch inertia: 88263.17581181342, ewa inertia: 88263.17581181342
Minibatc

C:\Users\Brech\anaconda3\envs\chidi\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 3072 or by setting the environment variable OMP_NUM_THREADS=9
  warnings.warn(


Init 1/3 with method k-means++
Inertia for init 1/3: 291209842.0
Init 2/3 with method k-means++
Inertia for init 2/3: 290578165.0
Init 3/3 with method k-means++
Inertia for init 3/3: 286756676.0
Minibatch step 1/100: mean batch inertia: 136860.92101483964
Minibatch step 2/100: mean batch inertia: 90905.13090136985, ewa inertia: 90905.13090136985
Minibatch step 3/100: mean batch inertia: 90207.23600475634, ewa inertia: 90207.23600475634
Minibatch step 4/100: mean batch inertia: 88983.0979907, ewa inertia: 88983.0979907
Minibatch step 5/100: mean batch inertia: 88463.5206334856, ewa inertia: 88463.5206334856
Minibatch step 6/100: mean batch inertia: 88240.42573966074, ewa inertia: 88240.42573966074
Minibatch step 7/100: mean batch inertia: 88736.16191746548, ewa inertia: 88736.16191746548
Minibatch step 8/100: mean batch inertia: 88078.33504280835, ewa inertia: 88078.33504280835
Minibatch step 9/100: mean batch inertia: 88120.18336865648, ewa inertia: 88120.18336865648
Minibatch step 10/

In [154]:
sample_feats_df = pd.DataFrame(sample_feats)

In [151]:
sample_feats

[array([0.06378738, 0.04186047, 0.03654485, 0.03189369, 0.03853821,
        0.01960133, 0.0372093 , 0.04418605, 0.0282392 , 0.03920266,
        0.01860465, 0.03853821, 0.02425249, 0.03289037, 0.02757475,
        0.03222591, 0.02591362, 0.04186047, 0.03189369, 0.02491694,
        0.02093023, 0.03122924, 0.03654485, 0.01594684, 0.04019934,
        0.02259136, 0.05282392, 0.03421927, 0.02425249, 0.04152824]),
 array([0.04551724, 0.03517241, 0.02275862, 0.04448276, 0.04758621,
        0.02551724, 0.02793103, 0.06689655, 0.01689655, 0.03206897,
        0.0337931 , 0.02551724, 0.03206897, 0.02275862, 0.04551724,
        0.04310345, 0.02689655, 0.02517241, 0.01896552, 0.02206897,
        0.03758621, 0.02655172, 0.04241379, 0.03413793, 0.03137931,
        0.03965517, 0.02862069, 0.02655172, 0.04172414, 0.03068966]),
 array([0.01561416, 0.02741152, 0.04510756, 0.02151284, 0.05898681,
        0.01873699, 0.04024983, 0.03990285, 0.04510756, 0.02637058,
        0.03469813, 0.04614851, 0.04649549, 

In [155]:
sample_feats_df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.063787,0.041860,0.036545,0.031894,0.038538,0.019601,0.037209,0.044186,0.028239,0.039203,...,0.020930,0.031229,0.036545,0.015947,0.040199,0.022591,0.052824,0.034219,0.024252,0.041528
1,0.045517,0.035172,0.022759,0.044483,0.047586,0.025517,0.027931,0.066897,0.016897,0.032069,...,0.037586,0.026552,0.042414,0.034138,0.031379,0.039655,0.028621,0.026552,0.041724,0.030690
2,0.015614,0.027412,0.045108,0.021513,0.058987,0.018737,0.040250,0.039903,0.045108,0.026371,...,0.023942,0.021860,0.035739,0.026718,0.029146,0.060722,0.025677,0.024983,0.015267,0.045455
3,0.030602,0.021761,0.017681,0.032982,0.030942,0.028222,0.034682,0.026862,0.066984,0.044203,...,0.026522,0.038422,0.046923,0.031282,0.031962,0.034682,0.017681,0.024141,0.068004,0.025162
4,0.045194,0.024283,0.023946,0.048229,0.030691,0.031703,0.035750,0.030354,0.026981,0.027993,...,0.033390,0.035076,0.022260,0.029005,0.033052,0.067454,0.027319,0.048904,0.029005,0.027993
5,0.030753,0.052522,0.022460,0.034900,0.044229,0.022460,0.024188,0.026261,0.031790,0.031099,...,0.044229,0.039737,0.036973,0.030408,0.027643,0.043538,0.027298,0.029026,0.021424,0.046303
6,0.029282,0.040518,0.031324,0.028941,0.040518,0.032346,0.059925,0.024855,0.025536,0.044263,...,0.048689,0.023153,0.023834,0.033027,0.035410,0.026217,0.041539,0.024515,0.043582,0.026558
7,0.035423,0.031173,0.021962,0.029756,0.024088,0.024442,0.040383,0.066596,0.031881,0.035069,...,0.019837,0.046050,0.084662,0.025859,0.030818,0.025859,0.024796,0.023734,0.027630,0.025151
8,0.023305,0.079802,0.026483,0.021540,0.027542,0.032486,0.033898,0.024011,0.046963,0.030720,...,0.024364,0.045198,0.044845,0.033192,0.025777,0.030720,0.029661,0.044138,0.022246,0.022246
9,0.025321,0.079529,0.028174,0.053138,0.031027,0.024251,0.037447,0.042796,0.033167,0.024608,...,0.023538,0.016762,0.024251,0.042083,0.039586,0.028531,0.036733,0.022825,0.041369,0.033524


Another thing I'll be doing to reduce the abuse on my CPU, is saving this sample in a separate csv-file in the data-folder. This way, I won't have to re-execute this operation over and over again.

In [156]:
sample_feats_df.to_csv('data/brecht-sample.csv')

In [162]:
from sklearn.model_selection import train_test_split

X = pd.read_csv("data/brecht-sample.csv")
Y = sample["choice"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

#### The Decision Tree Model

In [166]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree = dec_tree.fit(X_train, y_train)

In [171]:
dec_tree.score(X_test, y_test)

0.65

#### The Logistic Regression Model

In [168]:
from sklearn.linear_model import LogisticRegressionCV
log_reg = LogisticRegressionCV()
log_reg = log_reg.fit(X_train, y_train)

C:\Users\Brech\anaconda3\envs\chidi\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Brech\anaconda3\envs\chidi\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

In [170]:
log_reg.score(X_test, y_test)

0.75

#### The "Perceptotron" (non-deep neural network)

### Deep-learning neural networks

Any machine learning model is considered deep learning if it has at least three layers total. This includes input layers, output layers and the hidden layers in between.

When it comes to computer vision, neural networks have become the preferred option. I'll be making several neural networks with varying numbers of hidden layers. This is to test the impact an increasing number of layers has on the speed and performance of the model.

It wouldn't be an analysis if I didn't have an initial hypothesis: I'm going to start off from the viewpoint that the performance of our model will be directly proportional to the total number of hidden layers, while our speed will be inversely proportional to it. In other words: the more layers, the better our model performs, but the longer it will take to make a prediction. 

In [172]:
from sklearn.model_selection import train_test_split

#Well, what are we waiting for? Let's load in our data and get analyzing!
sample = os.listdir("data/images/3 De nayer (Loop piste)/partition 1")
sample = data.loc[data["image"].isin(sample)].head(50)

In [173]:
#Our X will contain the pixels of our loaded images, while our Y contains all of the classifiers
X = np.array([cv.imread("/".join(['data','images','3 De nayer (Loop piste)','partition 1',img])) for img in sample["image"]])
Y = np.array(sample["choice"])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

Our training set consists of 30 images, each of which consist of 480 x 640 BRG-pixels.

In [187]:
X_train.shape

(30, 480, 640, 3)

In [188]:
len(Y_train)

30

In [191]:
Y_train

array([2., 2., 2., 2., 0., 2., 2., 1., 2., 2., 0., 2., 1., 1., 0., 2., 2.,
       2., 0., 2., 2., 2., 2., 2., 1., 2., 0., 2., 1., 2.])

In [192]:
X_train = X_train / 255.0

In [195]:
X_test = X_test / 255.0

#### The models I made

* First Model: Includes only a Flatten-layer and a Dense-layer, the latter consisting of 3 neurons (as many as there are classes) and activated by the softmax-function.
* Second Model: Consists of 6 layers, including input and output, for a total of 4 hidden layers.
* Third Model: Consists of 8 layers, including input and output, for a total of 6 hidden layers.

In [196]:
first_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(480,640,3)),
    tf.keras.layers.Dense(3, activation='softmax')
])

second_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(480,640,3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

third_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(480,640,3)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='selu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='selu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In essence, my first model is nothing more than essentially our Logistic Regression-model from earlier converted into a neural network.

In [197]:
first_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

first_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 921600)            0         
                                                                 
 dense_22 (Dense)            (None, 3)                 2764803   
                                                                 
Total params: 2,764,803
Trainable params: 2,764,803
Non-trainable params: 0
_________________________________________________________________


In [210]:
first_model.fit(X_train, Y_train, epochs=25, batch_size=10, validation_split=0.2)

Epoch 1/25
3/3 [==============================] - 0s 144ms/step - loss: 2.3445 - accuracy: 0.7083 - val_loss: 14.2155 - val_accuracy: 0.5000
Epoch 2/25
3/3 [==============================] - 0s 130ms/step - loss: 9.3181 - accuracy: 0.7083 - val_loss: 8.3855 - val_accuracy: 0.5000
Epoch 3/25
3/3 [==============================] - 0s 99ms/step - loss: 5.0289 - accuracy: 0.8333 - val_loss: 19.7735 - val_accuracy: 0.1667
Epoch 4/25
3/3 [==============================] - 0s 96ms/step - loss: 5.0639 - accuracy: 0.5833 - val_loss: 12.5198 - val_accuracy: 0.5000
Epoch 5/25
3/3 [==============================] - 0s 108ms/step - loss: 3.8144 - accuracy: 0.6250 - val_loss: 12.8609 - val_accuracy: 0.5000
Epoch 6/25
3/3 [==============================] - 0s 111ms/step - loss: 8.7390 - accuracy: 0.6250 - val_loss: 8.6029 - val_accuracy: 0.3333
Epoch 7/25
3/3 [==============================] - 0s 110ms/step - loss: 9.2138 - accuracy: 0.5000 - val_loss: 21.2543 - val_accuracy: 0.5000
Epoch 8/25
3/3 [=

In [211]:
first_model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 70ms/step - loss: 5.4937 - accuracy: 0.8500


[5.493687629699707, 0.8500000238418579]

Our first model takes 74 milliseconds to make a prediction, with an accuracy of 80%. Performance-wise, it is better than even our Logistic Regression-model earlier. Considering this is supposed to be the "worst" deep learning model, I have high hopes for the rest of the models.

In [200]:
second_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

second_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 921600)            0         
                                                                 
 dense_23 (Dense)            (None, 128)               117964928 
                                                                 
 dense_24 (Dense)            (None, 32)                4128      
                                                                 
 dense_25 (Dense)            (None, 3)                 99        
                                                                 
Total params: 117,969,155
Trainable params: 117,969,155
Non-trainable params: 0
_________________________________________________________________


In [212]:
second_model.fit(X_train, Y_train, epochs=25, batch_size=10, validation_split=0.2)

Epoch 1/25
3/3 [==============================] - 4s 1s/step - loss: 0.7012 - accuracy: 0.7500 - val_loss: 1.3715 - val_accuracy: 0.5000
Epoch 2/25
3/3 [==============================] - 3s 1s/step - loss: 2.2412 - accuracy: 0.2083 - val_loss: 1.7491 - val_accuracy: 0.5000
Epoch 3/25
3/3 [==============================] - 4s 1s/step - loss: 2.2687 - accuracy: 0.5000 - val_loss: 5.1659 - val_accuracy: 0.5000
Epoch 4/25
3/3 [==============================] - 4s 1s/step - loss: 3.0736 - accuracy: 0.7083 - val_loss: 9.7794 - val_accuracy: 0.5000
Epoch 5/25
3/3 [==============================] - 4s 1s/step - loss: 3.9725 - accuracy: 0.7500 - val_loss: 7.4839 - val_accuracy: 0.5000
Epoch 6/25
3/3 [==============================] - 4s 1s/step - loss: 2.5652 - accuracy: 0.7083 - val_loss: 3.8156 - val_accuracy: 0.5000
Epoch 7/25
3/3 [==============================] - 4s 1s/step - loss: 2.3077 - accuracy: 0.5417 - val_loss: 6.3970 - val_accuracy: 0.5000
Epoch 8/25
3/3 [=========================

Training this model took considerably longer than our first model, with every epoch taking a handful of seconds to complete. The longest epoch in our last model lasted about one second.

In [213]:
second_model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 121ms/step - loss: 2.0548 - accuracy: 0.7500


[2.054837465286255, 0.75]

Speed seems to have gone down to 121 milliseconds per prediction made, but the model still has an accuracy 5% higher than our first neural network, and 10% higher than our Logistic Regression-model.

It does, however, depend on the data used. In a previous attempt, before the pixel-values of our images were converted to values between 0 and 1, our second model actually performed *worse* than not just our first model, but also every single one of our non-deep learning models. The performance of a neural network, therefore, is not just dependent on the structure of its layers, but also on the data fed into its inputs.

In [203]:
third_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

third_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_8 (Flatten)         (None, 921600)            0         
                                                                 
 dense_26 (Dense)            (None, 256)               235929856 
                                                                 
 dense_27 (Dense)            (None, 256)               65792     
                                                                 
 dense_28 (Dense)            (None, 128)               32896     
                                                                 
 dense_29 (Dense)            (None, 128)               16512     
                                                                 
 dense_30 (Dense)            (None, 64)                8256      
                                                                 
 dense_31 (Dense)            (None, 32)               

In [207]:
third_model.fit(X_train, Y_train, epochs=25, batch_size=10, validation_split=0.2)

Epoch 1/25
3/3 [==============================] - 8s 2s/step - loss: 8.7798 - accuracy: 0.6250 - val_loss: 8.7169 - val_accuracy: 0.5000
Epoch 2/25
3/3 [==============================] - 8s 3s/step - loss: 3.9329 - accuracy: 0.7083 - val_loss: 3.9863 - val_accuracy: 0.5000
Epoch 3/25
3/3 [==============================] - 8s 3s/step - loss: 2.3695 - accuracy: 0.6667 - val_loss: 14.9114 - val_accuracy: 0.1667
Epoch 4/25
3/3 [==============================] - 8s 3s/step - loss: 5.5436 - accuracy: 0.5833 - val_loss: 19.3375 - val_accuracy: 0.1667
Epoch 5/25
3/3 [==============================] - 8s 3s/step - loss: 8.3505 - accuracy: 0.4583 - val_loss: 23.5491 - val_accuracy: 0.5000
Epoch 6/25
3/3 [==============================] - 8s 3s/step - loss: 15.0465 - accuracy: 0.7083 - val_loss: 39.3249 - val_accuracy: 0.5000
Epoch 7/25
3/3 [==============================] - 8s 3s/step - loss: 20.6045 - accuracy: 0.3750 - val_loss: 30.2747 - val_accuracy: 0.5000
Epoch 8/25
3/3 [==================

In [208]:
third_model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 201ms/step - loss: 8.0819 - accuracy: 0.1500


[8.081899642944336, 0.15000000596046448]

Every prediction took about 169 milliseconds, and our accuracy has shot up to above 75%.

### Conclusions

* The way the data fed into a neural network is structured plays a big role in determining the overall accuracy of a neural network when classifying images.